# Basics Monte Carlo

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from ipywidgets import widgets, fixed

Recall that we are interested in computing 
$ C_0 = N_0 \mathbb{E}_Q[  C_T /N_T]$. 

Let us consider the Black-Scholes market, use $B$ as numéraire and consider a put option, so $C_0 = \mathbb{E}_Q [  \exp(-rT) \max(K-S_T, 0) ]$.

You have obtained an analytical formula, the famous Black-Scholes formula, during the tutorial. Suppose now that we were not able to do determine this expectation. Could we come up with an approximation?

Note that we have seen that it is quite easy to determine the distribution of $S_T$ under $\mathbb{Q}$. Indeed we have, under the above assumptions, 
$S_T=S_0\exp( (r-0.5\sigma^2)T + \sigma W_T^Q)$, where $W^Q$ is a standard Brownian motion under $Q$.  As we can simulate $W^Q_T$, and hence $S_T$, we can exploit Monte Carlo simulations to determine an approximation!





Please recall the (weak) law of large numbers from probability theory:  if $X_1,\dots,X_n$ are i.i.d. observations from a distribution with finite mean, then we have
$\bar{X}_n \stackrel{p}{\to} \mathbb{E}[X_1]$.


This leads to the following approach:


*   simulate $n$ draws of $S_T$ under $Q$: $S_T^{(1)},\dots,S_T^{(n)}$;
*   determine $X_i = \exp(-rT) \max(K - S_T^{(i)}, 0)$;
* approximate $C_0$ by $\bar{X}_n$.

According to the WLLN this gives us a good approximation if we use $n$ large enough. Let us see this in action.

First we calculate the price of the put using the (closed-form) Black-Scholes formula.




In [ ]:
def black_scholes_put(S, K, r, sigma, T):
  """Calculate price put option analytically.

  Parameters
  ----------
  S: float (strictly positive), current stock price;
  K: float (strictly positive), exercise price option;
  r: float (nonnegative), interest rate;
  sigma: float (strictly positive), volatility parameter in GBM;
  T: float (strictly positive), maturity
  """
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)
  price = np.exp(-r * T) * K * norm.cdf(- d2) - S * norm.cdf(- d1)
  print(f"The price of the put is {price}.")

# specify parameters:
S_0 = 100
K = 90
T = 2
sigma = 0.15
r = 0.01
black_scholes_put(S_0, K, r, sigma, T)

As an aside:

The following cell allows you to investigate the effect of the parameters on the price.


In [ ]:
S_0_config = np.linspace(100, 150, 3)
T_config = np.linspace(1, 2, 3)
K_config = np.linspace(90, 120, 3)
r_config = np.linspace(0.01, 0.03, 3)
sigma_config = np.linspace(0.1, 0.2, 3)
widgets.interact(black_scholes_put, S=S_0_config,  K=K_config,  r=r_config, sigma=sigma_config,  T=T_config)

Back to Monte Carlo! 

We now calculate a MC-approximation to the price of the put:

In [ ]:
def simulate_stock_maturity(n, T, S_0, r, sigma, seed=None):
  """Simulate S_T under the risk-neutral measure.

  Parameters
  ----------
  S_0: float (strictly positive), current stock price;
  r: float (nonnegative), interest rate;
  sigma: float (strictly positive), volatility parameter in GBM;
  T: float (strictly positive), maturity;
  n: integer, number of simulations.
  """
  W_T = np.sqrt(T) * norm.rvs(size=n, random_state=seed)
  return S_0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * W_T)

def MC_approximation_put(n, T, S_0, r, sigma, K, seed=None):
  """Simulate MC-approximation to price put.

  Parameters
  ----------
  S_0: float (strictly positive), current stock price;
  r: float (nonnegative), interest rate;
  sigma: float (strictly positive), volatility parameter in GBM;
  T: float (strictly positive), maturity;
  n: integer, number of simulations.
  """
  S = simulate_stock_maturity(n, T, S_0, r, sigma, seed)
  X = np.exp(-r * T) * np.maximum(K - S, np.zeros(n))
  return np.mean(X)
n = 5000
black_scholes_put(S_0, K, r, sigma, T)
print(f"MC approximation: {MC_approximation_put(n, T, S_0, r, sigma, K)}")

Run the above cell a view times to assess the quality of the approximation. And Inspect what happens if you increase $n$.

In practice we sometimes/often need that our results are reproducible (for example, for audit or validation purposes). Therefore, it can helpful to fix the seed of your random number generator. Check, by running the next cell a few times, that this indeed works. Warning: seeds are often relative to the specific version of your programming language and packages. Using seed=123 in Python 3.7 might give other results compared to using the same seed in Python 3.8.


In [ ]:
seed = 41
print(f"MC approximation: {MC_approximation_put(n, T, S_0, r, sigma, K, seed)}")

You have seen that increasing $n$ helps, for "most realizations", to get an approximation that is closer to the true price.

Can we also provide some theory for the quality of the approximation?  

Please recall that $\mathbb{E} \bar{X}_n = \mathbb{E} X_1$, so the MC-estimator is unbiased.
And we have $\operatorname{var}(\bar{X}_n) = \operatorname{var}(X_1) /n$. Moreover, the Central Limit Theorem gives us, for large $n$, an approximation to the distribution of the sample mean via  $ \sqrt{n}(\bar{X}_n - \mathbb{E}X_1 )\stackrel{d}{\approx} N(0, \operatorname{var}(X_1))$. This approximation leads to approximate confidence intervals:  $\bar{X}_n \pm \Phi^{-1}(1-\alpha/2) S_n /\sqrt{n}$, where $S_n$ is the sample standard deviation of $X_1,\dots,X_n$. This implies, for large $n$, $\mathbb{P}( \bar{X}_n - \Phi^{-1}(1-\alpha/2) S_n /\sqrt{n} \leq \mathbb{E}X_1\leq \bar{X}_n + \Phi^{-1}(1-\alpha/2) S_n /\sqrt{n})\approx 1- \alpha$.

Please note that we control $n$ in our setting! If we have some bound available on $S_n$, then we could calculate $n$ in such a way that a 95% confidence interval has, for example, a width of 1 cent.

Run the following cell a few times to inspect different realizations of the confidence interval. Also inspect the effect of increasing $n$.


In [ ]:
def MC_approximation_put_CI(n, T, S_0, r, sigma, K, alpha):
  """Determine (1-alpha)% confidence interval for the price of the put.""" 
  S = simulate_stock_maturity(n, T, S_0, r, sigma)
  X = np.exp(-r * T) * np.maximum(K - S, np.zeros(n))
  left = np.mean(X) - norm.ppf(1 - alpha / 2) * np.std(X) / np.sqrt(n)
  right = np.mean(X) + norm.ppf(1 - alpha / 2) * np.std(X) / np.sqrt(n)
  return left, right

black_scholes_put(S_0, K, r, sigma, T)
alpha = 0.05
n = 2500
print(f"An approximate 95% confidence interval for the price of the put is given by {MC_approximation_put_CI(n, T, S_0, r, sigma, K, alpha)}.")